<a href="https://colab.research.google.com/github/DavidWeaverAudio/StrayCompanionVOGenerator/blob/main/StrayCompanionVOGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **GENERATE COMPANION-ESQUE SPEECH FROM TEXT**
<img src="https://s.yimg.com/os/creatr-uploaded-images/2022-06/3cc4cb90-e6d7-11ec-bf95-e13207e799f9">
---
I absolutely fell in love with the sound of the companions in
[Stray](https://store.steampowered.com/app/1332010/Stray)
They have a super interesting voice, with a system I wanted to try to emulate here! 
<br>Note: I did NO work on this title, and I have no idea if this is the way they did it, but I highly encourage you to check out the sound designers;

*   Music Composer/Sound Designer : Yann Van Der Cruyssen [@Morusque](https://twitter.com/Morusque?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor)
*   Additional Sound Designer : Raphaël Monnin (Twitter handle not found)

Use this script to your heart's content, and generate some whacky companions!
In addition, play the incredible work of the team on Stray.
<br>To contact me, hit up david@weaveraudio.com or go to [weaveraudio.com](https://www.weaveraudio.com)

In [1]:
#@markdown ---
#@markdown ### Run this cell to import the necessary dependencies 
#@markdown ---
!pip install gtts
!pip install playsound
!pip install eng-to-ipa
!pip install librosa
!pip install nltk
import nltk
from nltk.corpus import wordnet
from gtts import gTTS
import os
import warnings
import playsound
import eng_to_ipa as ipa
import random
import librosa
import numpy as np
from numpy import concatenate
import soundfile as sf
from random import randrange
import ipywidgets as widgets
from IPython.display import display
from IPython.display import Audio


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7035 sha256=6392c483ca5d432d11a87c3211e640a385623c20d089378e29be5fbfeba1face
  Stored in directory: /root/.cache/pip/wheels/ba/f8/bb/ea57c0146b664dca3a0ada4199b0ecb5f9dfcb7b7e22b65ba2
Successfully built playsound
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 8.2 MB/s 
  Created wheel for eng-to-ipa: filename=eng_to_ipa-0.0.2-py3-none-any.whl size=2822641 sha256=28804bbb69301ed83890ac362f7b69aa51d968d32e48737bdf1a940c2cda47d9
  Stored in directory: /root/.cache/pip/wheels/96/c0/dd/aeddfbebc2c3301c3dd09670d9954b0574ac4cd982664c1110
Successfully built eng-to-ipa
Looking in indexes: https://pypi.org/simple, https://us-py

playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.
/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


In [2]:
#@markdown ---
#@markdown ### Run this cell to import the logic
#@markdown ---
nltk.download('wordnet')
nltk.download('omw-1.4')
pitch_amount = 0
stretch_amount = 1
last_pitch_shift_amount = 0

def Word_Count(str):
    counts = 0
    word_list = []
    words = str.split()
    for word in words:
        word_list.append(word)
        counts+= 1
    return counts,word_list

def IPA_Conversion(str):
    return ipa.convert(str)

def Re_Arrange_Words(str):
    sent = str
    sorted_arr = sent.split(" ")      
    sorted_arr.sort(reverse=True)
    return sorted_arr

def Get_Synonym(str):
  for syn in wordnet.synsets(str):  
    for name in syn.lemma_names():
        if(name):
          return name
        else:
          return ""

def Delete_Random_Words(list):
    new_list = []
    for elem in list:
      random_float = random.random()*10
      random_int = int(random_float)
      if(random_int > 3):
        new_list.append(elem)
    return new_list

def Replace_With_Synonyms(list):
    new_list = []
    for elem in list:
      new_element = Get_Synonym(elem)
      if(new_element != None):
        new_list.append(new_element)
      else:
        new_list.append(elem)
    return new_list
    
def Create_String_From_List(list, spaceStatus):
  new_string_from_list = ""
  for elem in list:
    if(space_status):
      new_string_from_list += elem + " "
    else:
      new_string_from_list += elem
  return new_string_from_list

def Create_Variation(iteration):
  original_string = text_block
  word_count, created_list = Word_Count(original_string)
  syno_replaced = Replace_With_Synonyms(created_list)

  text_to_speechify = IPA_Conversion(syno_replaced)

  text_list = Re_Arrange_Words(text_to_speechify)
  updated_list = Delete_Random_Words(text_list)
  current_word_index = 0
  arr_list = []
  while current_word_index < len(updated_list):
    resulting_np_array = Create_Audio(updated_list[current_word_index],current_word_index)
    current_word_index += 1
    arr_list.append(resulting_np_array)
  return arr_list

def Get_Pitch_Shift_Amount(pitch_amount):
  _pitch_shift_amount = random.randint(0, pitch_amount)
  if(random.randint(1,2) == 1):
    _pitch_shift_amount = -pitch_shift_amount
  return _pitch_shift_amount

def Get_Time_Stretch_Amount(stretch_amount,pitch_amount):
  time_stretch_amount = random.randint(1,stretch_amount)
  if(time_stretch_amount < 1):
    time_stretch_amount = 0.2
  return time_stretch_amount

def Trim_silence(audio_file):
  yt, index = librosa.effects.trim(audio_file)
  return yt

def Pitch_shift_Audio(audio_file):
  shifted = librosa.effects.pitch_shift(audio_file, sr=44100, n_steps=flat_pitch_change)
  return shifted

def Create_Audio(word, word_index):
  word_index_label = str(word_index)
  sound_file_output = gTTS(text=word, lang=language, slow=True)
  sound_file_output.save(filename + "_" + word_index_label + ".wav")
  os.system("mpg321 " + filename + "_" + word_index_label + ".wav")
  y, sr = librosa.load(filename +"_" + word_index_label + ".wav", sr=44100) 
  pitch_shift_amount = Get_Pitch_Shift_Amount(pitch_amount)
  time_stretch_amount = Get_Time_Stretch_Amount(stretch_amount,pitch_shift_amount)
  y_shifted = librosa.effects.pitch_shift(y, sr, n_steps=pitch_shift_amount)
  y_fast = librosa.effects.time_stretch(y_shifted, time_stretch_amount) 
  return y_fast

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


## **Creating your companion voice**

In [8]:
iteration = 0
output_list = []
#@markdown ---
#@markdown ### Name of output file
file_name = 'companion_robot_voice' #@param {type:"string"}
amount_to_create =  3#@param {type:"number"}
#@markdown ### Text input, works best with ~10 words

text_block = "Lost, alone and separated from family, a stray cat must untangle an ancient mystery to escape a long-forgotten cybercity and find their way home." #@param {type:"string"}
language_selection = "French (fr)"#@param ["Afrikaans (af)", "Albanian (sq)","Arabic (ar)","Bengali (bn)", "Bosnian (bs)", "Bulgarian (bg)", "Catalan (ca)", "Chinese Simplified (zh-CN)", "Chinese Traditional (zh-TW)", "Croatian (hr)", "Czech (cs)", "Danish (da)", "Dutch (nl)","English (en)","Filipino (tl)","Finnish (fi)","French (fr)","German (de)","Greek (el)","Gujarati (gu)","Hebrew (iw)","Hindi (hi)","Hungarian (hu)","Icelandic (is)","Indonesian (id)","Italian (it)","Japanese (ja)","Khmer (km)","Korean (ko)","Latvian (lv)","Lithuanian (lt)","Malay (ms)","Malayalam (ml)","Nepali (ne)","Norwegian (no)","Polish (pl)","Romanian (ro)","Russian (ru)","Spanish (es)","Swedish (sv)","Tamil (ta)","Telugu (te)","Thai (th)","Turkish (tr)","Ukrainian (uk)","Urdu (ur)","Vietnamese (vi)"]
pitch_shift_amount =  10#@param {type:"number"}
random_pitch_shift_amount =  4#@param {type:"number"}
time_stretch_amount =  12#@param {type:"number"}

#@markdown ---
startingLangCode = language_selection.find("(")
endingLangCode = language_selection.find(")")
language = ""
for x in range(startingLangCode+1,endingLangCode):
 language += language_selection[x]
pitch_amount = random_pitch_shift_amount
flat_pitch_change = pitch_shift_amount

stretch_amount = time_stretch_amount
print("Successfully set up your audio")

Successfully set up your audio


In [9]:
import warnings
warnings.filterwarnings('ignore')
#@markdown ---
#@markdown ### Run this cell to create your variations!
#@markdown ---
for x in range(amount_to_create):
  results = Create_Variation(str(iteration))
  iteration +=1
  num_arrays_to_conc = len(results)
  output= np.concatenate(results)
  output = Trim_silence(output)
  output = Pitch_shift_Audio(output)
  sf.write(filename + ".wav", output, 44100, 'PCM_16')
  output_audio = Audio(filename + ".wav", autoplay=True)
  display(output_audio)

